In [1]:
from IPython.core.magic import Magics, cell_magic, line_magic, magics_class
from pexpect import spawn

TIMEOUT = 300
PROG = "hive"
PROMPT = ["\r\n    > ", "\r\nhive> "]
QUIT = "quit;"


@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def hive(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace("\r\n", "\n")
            output = output.split("\n")
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    print(output_line)
        return None

    @line_magic
    def quit(self, line):
        self.app.sendline(QUIT)


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [7]:
!hdfs dfs -copyFromLocal data.tsv /tmp

In [8]:
!hdfs dfs -ls /tmp

Found 3 items
-rw-r--r--   1 root supergroup        677 2023-05-28 20:04 /tmp/data.tsv
drwxrwx---   - root supergroup          0 2023-05-28 19:55 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2023-05-28 19:57 /tmp/hive


In [9]:
%%hive
DROP TABLE IF EXISTS data;

TIMEOUT: Timeout exceeded.
<pexpect.pty_spawn.spawn object at 0x7f08e035e640>
command: /opt/hive/bin/hive
args: ['/opt/hive/bin/hive']
buffer (last 100 chars): b'ation_1685303719471_0001/\r\nKill Command = /opt/hadoop/bin/hadoop job  -kill job_1685303719471_0001\r\n'
before (last 100 chars): b'ation_1685303719471_0001/\r\nKill Command = /opt/hadoop/bin/hadoop job  -kill job_1685303719471_0001\r\n'
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 1724
child_fd: 64
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 2000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0.05
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_re:
    0: re.compile(b'\r\n    > ')
    1: re.compile(b'\r\nhive> ')

In [5]:
%%hive
CREATE TABLE data (
letra STRING,
fecha STRING,
valor INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY  '\t';

OK
Time taken: 0.889 seconds


In [6]:
%%hive
LOAD DATA 'data.tsv' OVERWRITE INTO TABLE data;
SELECT * FROM data;

Loading data to table default.data
OK
Time taken: 1.529 seconds
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
E	1994-02-14	5
A	1988-04-27	12
A	1990-10-06	10
E	1985-02-12	16
E	1998-09-14	16
B	1994-08-30	17
A	1997-12-15	13
B	1995-08-23	10
B	1998-11-22	13
B	1997-04-09	14
E	1993-12-27	18
E	1999-01-14	15
A	1992-09-19	18
B	1993-03-02	14
B	1999-10-21	13
A	1990-08-31	12
C	1994-01-25	6
E	1990-02-09	18
A	1990-09-26	14
A	1993-05-08	16
B	1995-09-06	14
E	1991-02-18	14
A	1993-01-11	14
A	1990-07-22	18
C	1994-09-09	15
C	1994-07-27	7
D	1990-10-10	15
A	1990-09-05	11
B	1991-10-01	15
A	1994-10-25	13
Time taken: 1.363 seconds, Fetched: 40 row(s)


In [6]:
%%hive
LOAD DATA INPATH '/tmp/data.tsv' OVERWRITE INTO TABLE data;
SELECT letra, COUNT(*) cantidad
FROM data
GROUP BY letra;

Loading data to table default.data
OK
Time taken: 1.257 seconds


TIMEOUT: Timeout exceeded.
<pexpect.pty_spawn.spawn object at 0x7f08e035e640>
command: /opt/hive/bin/hive
args: ['/opt/hive/bin/hive']
buffer (last 100 chars): b'ation_1685303719471_0001/\r\nKill Command = /opt/hadoop/bin/hadoop job  -kill job_1685303719471_0001\r\n'
before (last 100 chars): b'ation_1685303719471_0001/\r\nKill Command = /opt/hadoop/bin/hadoop job  -kill job_1685303719471_0001\r\n'
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 1724
child_fd: 64
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 2000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0.05
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_re:
    0: re.compile(b'\r\n    > ')
    1: re.compile(b'\r\nhive> ')

In [34]:
%%hive
SELECT * FROM data;

FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [ ]:
%%hive
show tables;

In [33]:
%%hive
SHOW TABLES;

SyntaxError: invalid syntax (888657108.py, line 1)

In [36]:
quit